In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s4e1/train.csv")

In [3]:
def transform(x):
    return x.apply(lambda y:1 if y == True else 0)

In [4]:
train.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [5]:
# new = pd.DataFrame(train.groupby(by = ['Geography','Exited'])['Exited'].count()/len(train_new)).unstack(level = -1).reset_index()
# new.columns = new.columns.map(lambda x: x[1])
# new = new.rename({'':'Geography'}, axis = 1)

In [6]:
def trans(df):
    if df == 'train':
        train_new = train.drop(['CustomerId','Surname','id'], axis= 1)
        train_new['Tenure'] = train_new['Tenure'].astype('category')
        train_new['NumOfProducts'] = train_new['NumOfProducts'].astype('category')
        train_new = pd.get_dummies(train_new)
        train_new_normal = train_new[['CreditScore', 'Age', 'Balance', 'HasCrCard', 'IsActiveMember',
           'EstimatedSalary', 'Exited']]
        train_new_ord = train_new[['Geography_France', 'Geography_Germany',
           'Geography_Spain', 'Gender_Female', 'Gender_Male', 'Tenure_0',
           'Tenure_1', 'Tenure_2', 'Tenure_3', 'Tenure_4', 'Tenure_5', 'Tenure_6',
           'Tenure_7', 'Tenure_8', 'Tenure_9', 'Tenure_10', 'NumOfProducts_1',
           'NumOfProducts_2', 'NumOfProducts_3', 'NumOfProducts_4']]
        train_new_ord = train_new_ord.apply(lambda x:transform(x))
        train_new = pd.concat([train_new_normal,train_new_ord], axis = 1)
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        x = sc.fit_transform(train_new.drop('Exited', axis = 1))
        y = train_new[['Exited']]
        return x,y
    else:
        test_new = test.drop(['CustomerId','Surname','id'], axis= 1)
        test_new['Tenure'] = test_new['Tenure'].astype('category')
        test_new['NumOfProducts'] = test_new['NumOfProducts'].astype('category')
        test_new = pd.get_dummies(test_new)
        test_new_normal = test_new[['CreditScore', 'Age', 'Balance', 'HasCrCard', 'IsActiveMember',
           'EstimatedSalary']]
        test_new_ord = test_new[['Geography_France', 'Geography_Germany',
           'Geography_Spain', 'Gender_Female', 'Gender_Male', 'Tenure_0',
           'Tenure_1', 'Tenure_2', 'Tenure_3', 'Tenure_4', 'Tenure_5', 'Tenure_6',
           'Tenure_7', 'Tenure_8', 'Tenure_9', 'Tenure_10', 'NumOfProducts_1',
           'NumOfProducts_2', 'NumOfProducts_3', 'NumOfProducts_4']]
        test_new_ord = test_new_ord.apply(lambda x:transform(x))
        test_new = pd.concat([test_new_normal,test_new_ord], axis = 1)
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        x = sc.fit_transform(test_new)
        return x

In [7]:
p,py = trans('train')

In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 400,max_features = "sqrt",oob_score = True,random_state = 3,verbose = 1,class_weight = 'balanced_subsample')

In [9]:
rf = rf.fit(p,py)

/tmp/ipykernel_18/3911038217.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = rf.fit(p,py)
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   18.2s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:  1.2min


In [10]:
test = pd.read_csv("/kaggle/input/playground-series-s4e1/test.csv")

In [11]:
test = trans('test')

In [12]:
d = rf.predict(test)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    1.8s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    7.3s
